In [3]:
slots_data = {
    '0' : "8:00 - 9:20",
    '000': "8:00 - 9:20",
    '001': "9:35 - 10:55",
    '010': "11:10 -12:30",
    '011': "12:45 - 2:05",
    '100': "2:20 - 3:40",
    '101': "3:55 - 5:15"
}

rooms_data = {
    '000': "C-401",     # classrooms - 60 strength
    '001': "C-403",
    '010': "C-405",
    '011': "C-406",
    '100': "C-402",     # halls - 120 strength
    '101': "C-404",
    '110': "Rawal-1",   # labs
    '111': "Rawal-2"
}

profs_data = {
    '00': "Mr. Shehryaar Rashid",
    '01': "Mr. Usman Ashraf",
    '10': "Ms. Bushra"
}

courses_data = {
    '000': "PF",
    '001': "COAL",
    '010': "Discrete Structures",
    '011': "Calculus",
    '100': "PF lab",
    '101': "COAL lab",
    '110': "Pakistan Studies",
    '111': "AP"
}

sections_data = {
    '0000': 'Section A',
    '0001': 'Section B',
    '0010': 'Section C',
    '0011': 'Section D',
    '0100': 'Section E',
    '0101': 'Section F',
    '0110': 'Section G',
    '0111': 'Section H',
    '1000': 'Section I',
    '1001': 'Section J',
    '1010': 'Section K',
    '1011': 'Section L',
    '1100': 'Section M',
    '1101': 'Section N',
    '1110': 'Section R',
    '1111': 'Section Z'
}

days_data = {
    '000': "Monday",
    '001': "Tuesday",
    '010': "Wednesday",
    '011': "Thursday",
    '100': "Friday",
    '110': "Saturday"
}

# Population/Chromosome (Timetable) Generation

In [ ]:
import random

courses = 8
sections = 16
rooms = 8
profs = 3
slots = 6
days = 5
max_strength = 120


def room_occupied(chromosome, day, timeslot, room):
    for course_info in chromosome:
        # print("Course Info:", course_info)
        for i in range(4, len(course_info), 4):
            if i + 2 < len(course_info):  # Check if the index is within bounds
                if course_info[i] == day and course_info[i + 1] == timeslot and int(course_info[i + 2], 2) == room:
                    return True
    return False


def generate_chromosome():
    chromosome = []

    # Generate a list of available courses
    available_courses = [format(i, '03b') for i in range(courses)]

    for section in range(sections):
        section_courses = random.sample(available_courses, 5)  # Select 5 distinct random courses for this section

        for course in section_courses:
            course_info = []

            # Generating course info
            course_info.append(course)  # Course (3 bits)
            course_info.append(str(random.randint(0, 1)))  # Theory/Lab (1 bit)
            course_info.append(format(section, '04b'))  # Section (4 bits)
            course_info.append(format(random.randint(0, max_strength - 1), '07b'))  # Section Strength (7 bits)
            course_info.append(format(random.randint(0, profs - 1), '02b'))  # Professor (2 bits)

            # Initialize lecture info placeholders
            lectures = []

            for _ in range(2):  # Two lectures per week
                while True:
                    lecture_info = []

                    # Generating lecture info
                    day = format(random.randint(0, days - 1), '03b')  # Day (3 bits)
                    timeslot = format(random.randint(0, slots - 1), '03b')  # Timeslot (3 bits)

                    if course_info[1] == '0':  # If it's a theory course
                        day = format(random.randint(0, days - 2), '03b')  # Avoiding last day for theory classes

                    lecture_info.append(day)
                    lecture_info.append(timeslot)

                    if course_info[1] == '1':  # If it's a lab course
                        # Lab lectures should be conducted in two consecutive slots
                        if int(timeslot, 2) == slots - 1:  # If last timeslot, adjust to ensure consecutive slots
                            timeslot = format(int(timeslot, 2) - 1, '03b')
                        lecture_info.append(timeslot)
                        lecture_info.append(timeslot)
                    else:
                        lecture_info.append(format(random.randint(0, rooms - 1), '03b'))  # Room (3 bits)
                        lecture_info.append(str(random.randint(0, 1)))  # Room size (1 bit)

                    # Check hard constraints
                    conflicting = False
                    for existing_lecture in lectures:
                        if lecture_info[:2] == existing_lecture[:2]:  # Check if day and timeslot are same
                            conflicting = True
                            break

                    # Check if room is free
                    if not conflicting and course_info[1] == '0':  # Check room availability for theory classes
                        room = int(lecture_info[2], 2)
                        if room_occupied(chromosome, day, timeslot, room):
                            conflicting = True

                    if not conflicting:
                        lectures.append(lecture_info)
                        break

            # Add lectures to course info
            for lecture in lectures:
                course_info.extend(lecture)
            chromosome.append(course_info)

    return chromosome

# Fitness Function

In [ ]:
def calculate_fitness(chromosome):
    conflicts = 0

    # Initialize a dictionary to store the schedule of each room
    room_schedule = {room: set() for room in range(rooms)}

    # Initialize a dictionary to store the schedule of each professor
    professor_schedule = {prof: set() for prof in range(profs)}

    # Initialize a dictionary to store the number of courses assigned to each professor
    professor_courses = {prof: 0 for prof in range(profs)}

    # Initialize a dictionary to store the schedule of each section
    section_schedule = {section: set() for section in range(sections)}

    # Initialize a dictionary to store the number of courses assigned to each section
    section_courses = {section: 0 for section in range(sections)}

    # Iterate through each course in the chromosome
    for course_info in chromosome[:]:  # Exclude the fitness value from the iteration
        # Get the section of the course
        section = int(course_info[2], 2)

        # Check if the section has already reached the course limit
        if section_courses[section] >= 5:
            conflicts += 1  # Penalize if the section takes up more than 5 courses
        else:
            section_courses[section] += 1

            # Initialize a list to store the days on which lectures are scheduled for the current course
            lecture_days = []

            # Initialize a flag to track if lab lectures are consecutive
            lab_consecutive = False

            # Iterate through each lecture of the course
            for i in range(4, len(course_info), 4):
                # Check if the length of course_info is valid for accessing indices
                if i + 3 < len(course_info):
                    day = int(course_info[i], 2)
                    timeslot = int(course_info[i + 1], 2)
                    room = int(course_info[i + 2], 2)
                    professor = int(course_info[4], 2)  # Professor assigned to the lecture

                    # Check if the professor is available at the specified time slot
                    if (day, timeslot) in professor_schedule[professor]:
                        conflicts += 1
                    else:
                        professor_schedule[professor].add((day, timeslot))

                    # Check if the room is occupied at the specified time slot
                    if room_occupied(chromosome[:], day, timeslot, room):  # Pass the chromosome without the fitness value
                        conflicts += 1

                    # Check if the room is available (Hard Constraint 1)
                    if room == 0 and room_occupied(chromosome[:], day, timeslot, 0):  # Pass the chromosome without the fitness value
                        conflicts += 1

                    # Check room size constraint (Hard Constraint 2)
                    section_strength = int(course_info[3], 2)
                    room_size = int(course_info[i + 3], 2)
                    if room_size == 0:  # Classroom size
                        if section_strength > 60:  # Assuming classroom capacity of 60
                            conflicts += 1
                    elif room_size == 1:  # Large hall size
                        if section_strength > 120:  # Assuming large hall capacity of 120
                            conflicts += 1

                    # Check Section-Room constraint 4
                    if (day, timeslot) in room_schedule[room]:
                        conflicts += 1
                    else:
                        room_schedule[room].add((day, timeslot))

                    # Check Room-Section constraint 5
                    if (day, timeslot) in section_schedule[section]:
                        conflicts += 1
                    else:
                        section_schedule[section].add((day, timeslot))

                    # Check Professor Course Limit Constraint 3
                    if professor_courses[professor] >= 3:
                        conflicts += 1
                    else:
                        professor_courses[professor] += 1

                    # Check Lab Lecture Constraint 9
                    if course_info[1] == '1':  # Lab course
                        if lab_consecutive:  # Check if previous lecture was a lab lecture
                            if timeslot != prev_timeslot + 1:  # Check if current timeslot is consecutive
                                conflicts += 1
                        else:
                            lab_consecutive = True
                    else:
                        lab_consecutive = False

                    # Update previous timeslot
                    prev_timeslot = timeslot

                    # Check Lecture Frequency Constraint
                    if day in lecture_days:
                        conflicts += 1
                    else:
                        lecture_days.append(day)
                        if len(lecture_days) > 2 or (len(lecture_days) == 2 and abs(lecture_days[0] - lecture_days[1]) <= 1):
                            conflicts += 1

    #chromosome[-1] = -conflicts
    # Return the inverse of conflicts as fitness value
    return -conflicts


# Tournament Selection

In [ ]:
from random import randint

def tournament_selection(population, fitness_scores, tournament_size):
    num_parents = len(population) // tournament_size
    parents = []

    for _ in range(num_parents):
        # Randomly select tournament participants
        tournament_participants = [randint(0, len(population)-1) for _ in range(tournament_size)]

        # Find the fittest individual in the tournament
        winner = max(tournament_participants, key=lambda x: fitness_scores[x])

        # Add the winner to the list of parents
        parents.append(population[winner])

    return parents


# Crossover Function

In [ ]:
def one_point_crossover(parent1, parent2):
    # Select a random crossover point
    crossover_point = randint(1, len(parent1) - 1)

    # Perform crossover
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]

    return child1, child2


# Fitness Function

In [ ]:
def calculate_fitness(chromosome):
    conflicts = 0

    # Initialize a dictionary to store the schedule of each room
    room_schedule = {room: set() for room in range(rooms)}

    # Initialize a dictionary to store the schedule of each professor
    professor_schedule = {prof: set() for prof in range(profs)}

    # Initialize a dictionary to store the number of courses assigned to each professor
    professor_courses = {prof: 0 for prof in range(profs)}

    # Initialize a dictionary to store the schedule of each section
    section_schedule = {section: set() for section in range(sections)}

    # Initialize a dictionary to store the number of courses assigned to each section
    section_courses = {section: 0 for section in range(sections)}

    # Iterate through each course in the chromosome
    for course_info in chromosome[:]:  # Exclude the fitness value from the iteration
        # Get the section of the course
        section = int(course_info[2], 2)

        # Check if the section has already reached the course limit
        if section_courses[section] >= 5:
            conflicts += 1  # Penalize if the section takes up more than 5 courses
        else:
            section_courses[section] += 1

            # Initialize a list to store the days on which lectures are scheduled for the current course
            lecture_days = []

            # Initialize a flag to track if lab lectures are consecutive
            lab_consecutive = False

            # Iterate through each lecture of the course
            for i in range(4, len(course_info), 4):
                # Check if the length of course_info is valid for accessing indices
                if i + 3 < len(course_info):
                    day = int(course_info[i], 2)
                    timeslot = int(course_info[i + 1], 2)
                    room = int(course_info[i + 2], 2)
                    professor = int(course_info[4], 2)  # Professor assigned to the lecture

                    # Check if the professor is available at the specified time slot
                    if (day, timeslot) in professor_schedule[professor]:
                        conflicts += 1
                    else:
                        professor_schedule[professor].add((day, timeslot))

                    # Check if the room is occupied at the specified time slot
                    if room_occupied(chromosome[:], day, timeslot, room):  # Pass the chromosome without the fitness value
                        conflicts += 1

                    # Check if the room is available (Hard Constraint 1)
                    if room == 0 and room_occupied(chromosome[:], day, timeslot, 0):  # Pass the chromosome without the fitness value
                        conflicts += 1

                    # Check room size constraint (Hard Constraint 2)
                    section_strength = int(course_info[3], 2)
                    room_size = int(course_info[i + 3], 2)
                    if room_size == 0:  # Classroom size
                        if section_strength > 60:  # Assuming classroom capacity of 60
                            conflicts += 1
                    elif room_size == 1:  # Large hall size
                        if section_strength > 120:  # Assuming large hall capacity of 120
                            conflicts += 1

                    # Check Section-Room constraint 4
                    if (day, timeslot) in room_schedule[room]:
                        conflicts += 1
                    else:
                        room_schedule[room].add((day, timeslot))

                    # Check Room-Section constraint 5
                    if (day, timeslot) in section_schedule[section]:
                        conflicts += 1
                    else:
                        section_schedule[section].add((day, timeslot))

                    # Check Professor Course Limit Constraint 3
                    if professor_courses[professor] >= 3:
                        conflicts += 1
                    else:
                        professor_courses[professor] += 1

                    # Check Lab Lecture Constraint 9
                    if course_info[1] == '1':  # Lab course
                        if lab_consecutive:  # Check if previous lecture was a lab lecture
                            if timeslot != prev_timeslot + 1:  # Check if current timeslot is consecutive
                                conflicts += 1
                        else:
                            lab_consecutive = True
                    else:
                        lab_consecutive = False

                    # Update previous timeslot
                    prev_timeslot = timeslot

                    # Check Lecture Frequency Constraint
                    if day in lecture_days:
                        conflicts += 1
                    else:
                        lecture_days.append(day)
                        if len(lecture_days) > 2 or (len(lecture_days) == 2 and abs(lecture_days[0] - lecture_days[1]) <= 1):
                            conflicts += 1

    #chromosome[-1] = -conflicts
    # Return the inverse of conflicts as fitness value
    return -conflicts


# Tournament Selection

In [ ]:
from random import randint

def tournament_selection(population, fitness_scores, tournament_size):
    num_parents = len(population) // tournament_size
    parents = []

    for _ in range(num_parents):
        # Randomly select tournament participants
        tournament_participants = [randint(0, len(population)-1) for _ in range(tournament_size)]

        # Find the fittest individual in the tournament
        winner = max(tournament_participants, key=lambda x: fitness_scores[x])

        # Add the winner to the list of parents
        parents.append(population[winner])

    return parents


# Crossover Function

In [ ]:
def one_point_crossover(parent1, parent2):
    # Select a random crossover point
    crossover_point = randint(1, len(parent1) - 1)

    # Perform crossover
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]

    return child1, child2


# Mutation Function

In [ ]:
def mutate(chromosome, mutation_rate):
    mutated_chromosome = chromosome.copy()

    for course_info in mutated_chromosome:
        # Apply mutation with a certain probability (mutation_rate)
        if random.random() < mutation_rate:
            # Choose a random attribute to mutate
            attribute_index = random.randint(0, len(course_info) - 1)

            # Mutate the selected attribute based on its type
            if attribute_index == 0:  # Course
                mutated_course = format(random.randint(0, courses - 1), '03b')
                course_info[attribute_index] = mutated_course
            elif attribute_index == 1:  # Theory/Lab
                mutated_type = str(random.randint(0, 1))
                course_info[attribute_index] = mutated_type
            elif attribute_index == 2:  # Section
                mutated_section = format(random.randint(0, sections - 1), '04b')
                course_info[attribute_index] = mutated_section
            elif attribute_index == 3:  # Section Strength
                mutated_strength = format(random.randint(0, max_strength - 1), '07b')
                course_info[attribute_index] = mutated_strength
            elif attribute_index == 4:  # Professor
                mutated_professor = format(random.randint(0, profs - 1), '02b')
                course_info[attribute_index] = mutated_professor
            elif attribute_index in range(5, len(course_info), 4):  # Lecture attributes
                if attribute_index + 3 < len(course_info):  # Check if the index is within bounds
                    if course_info[1] == '1':  # Lab course
                        # Mutate room and room size together for lab courses
                        mutated_room = format(random.randint(0, rooms - 1), '03b')
                        mutated_room_size = str(random.randint(0, 1))
                        course_info[attribute_index] = mutated_room
                        course_info[attribute_index + 1] = mutated_room_size
                    else:
                        # Mutate day, timeslot, and room individually for theory courses
                        if attribute_index % 4 == 1:  # Day
                            mutated_day = format(random.randint(0, days - 2), '03b')
                            course_info[attribute_index] = mutated_day
                        elif attribute_index % 4 == 2:  # Timeslot
                            mutated_timeslot = format(random.randint(0, slots - 1), '03b')
                            course_info[attribute_index] = mutated_timeslot
                        elif attribute_index % 4 == 3:  # Room
                            mutated_room = format(random.randint(0, rooms - 1), '03b')
                            course_info[attribute_index] = mutated_room

    return mutated_chromosome


# Time Table Scheduling - Genetic Algorithm Application

In [ ]:
population_size = 1250
tournament_size = 5
mutation_rate = 0.2
generations = 0

# Generate initial population
chromosomes = [generate_chromosome() for _ in range(population_size)]


In [ ]:
# Iterate until only one chromosome left
while len(chromosomes) > 1:
    fitness = [calculate_fitness(chromosome) for chromosome in chromosomes]
    if(len(chromosomes) >= tournament_size):
        # Selection
        selected_parents = tournament_selection(chromosomes, fitness, tournament_size)

        # Crossover
        new_generation = []
        for i in range(0, len(selected_parents), 2):
            child1, child2 = one_point_crossover(selected_parents[i], selected_parents[i+1])
            new_generation.extend([child1, child2])

        # Mutation
        for i in range(len(new_generation)):
            if random.random() < mutation_rate:
                new_generation[i] = mutate(new_generation[i], mutation_rate)

        chromosomes = new_generation
        generations += 1

        print("Generation:", generations)
        print("Population size:", len(chromosomes))
    else:
        #chromosomes.sort()
        # Combine chromosomes and fitness scores into tuples
        chromosomes_with_fitness = list(zip(chromosomes, fitness))
        # Sort the tuples based on fitness scores (in descending order)
        sorted_chromosomes_with_fitness = sorted(chromosomes_with_fitness, key=lambda x: x[1], reverse=True)
        # Separate sorted chromosomes and fitness scores
        sorted_chromosomes, sorted_fitness = zip(*sorted_chromosomes_with_fitness)
        break

for i in sorted_chromosomes:
    for ii in i:
        print(ii)
    print("\n")

Generation: 1
Population size: 250
Generation: 2
Population size: 50
Generation: 3
Population size: 10
Generation: 4
Population size: 2
['101', '1', '0000', '1011100', '10', '100', '001', '001', '001', '100', '010', '010', '010']
['000', '0', '0000', '0010101', '10', '011', '001', '100', '1', '001', '001', '001', '1']
['010', '0', '0000', '0110100', '10', '011', '101', '010', '0', '011', '000', '101', '1']
['001', '1', '0000', '0100001', '10', '010', '001', '001', '001', '100', '010', '010', '010']
['011', '1', '0000', '0000011', '10', '000', '011', '011', '011', '000', '010', '010', '010']
['010', '0', '0001', '0100110', '00', '001', '001', '010', '1', '010', '010', '000', '0']
['101', '0', '0001', '0110110', '01', '011', '100', '100', '0', '011', '001', '101', '0']
['111', '0', '0001', '0000010', '00', '010', '101', '111', '1', '011', '010', '000', '0']
['011', '1', '0001', '1000100', '01', '000', '100', '100', '100', '010', '100', '100', '100']
['000', '0', '0001', '1011010', '10', 

In [ ]:
decoded_chromosome = []

for chromosome in sorted_chromosomes[0]:
    decoded_chromosome_entry = []
    print(chromosome)
    # Decoding Course
    course = courses_data[chromosome[0]]
    decoded_chromosome_entry.append(course)

    # Decoding Theory/Lab
    theory_lab = "Theory" if chromosome[1] == '0' else "Lab"
    decoded_chromosome_entry.append(theory_lab)

    # Decoding Section
    section = sections_data[chromosome[2]]
    decoded_chromosome_entry.append(section)

    # Decoding Section Strength
    section_strength = int(chromosome[3], 2)
    decoded_chromosome_entry.append(section_strength)

    # Decoding Professor
    professor = profs_data[chromosome[4]]
    decoded_chromosome_entry.append(professor)
    print(chromosome[5])
    print(days_data[chromosome[5]])
    # Decoding First Lecture Day
    first_lecture_day = days_data[chromosome[5]]
    decoded_chromosome_entry.append(first_lecture_day)

    # Decoding First Lecture Time Slot
    first_lecture_time_slot = slots_data[chromosome[6]]
    decoded_chromosome_entry.append(first_lecture_time_slot)

    # Decoding First Lecture Room
    first_lecture_room = rooms_data[chromosome[7]]
    decoded_chromosome_entry.append(first_lecture_room)

    # Decoding First Lecture Room Size
    first_lecture_room_size = "60" if chromosome[8] == '0' else "120"
    decoded_chromosome_entry.append(first_lecture_room_size)

    print(chromosome[9])
    # Decoding Second Lecture Day
    second_lecture_day = days_data[chromosome[9]]
    decoded_chromosome_entry.append(second_lecture_day)
    print(chromosome[10])
    # Decoding Second Lecture Time Slot
    second_lecture_time_slot = slots_data[chromosome[10]]
    decoded_chromosome_entry.append(second_lecture_time_slot)

    # Decoding Second Lecture Room
    second_lecture_room = rooms_data[chromosome[11]]
    decoded_chromosome_entry.append(second_lecture_room)

    # Decoding Second Lecture Room Size
    second_lecture_room_size = "60" if chromosome[12] == '0' else "120"
    decoded_chromosome_entry.append(second_lecture_room_size)

    decoded_chromosome.append(decoded_chromosome_entry)

# Printing the decoded chromosome
for entry in decoded_chromosome:
    print(entry)
print(decoded_chromosome)

['101', '1', '0000', '1011100', '10', '100', '001', '001', '001', '100', '010', '010', '010']
100
Friday
100
010
['000', '0', '0000', '0010101', '10', '011', '001', '100', '1', '001', '001', '001', '1']
011
Thursday
001
001
['010', '0', '0000', '0110100', '10', '011', '101', '010', '0', '011', '000', '101', '1']
011
Thursday
011
000
['001', '1', '0000', '0100001', '10', '010', '001', '001', '001', '100', '010', '010', '010']
010
Wednesday
100
010
['011', '1', '0000', '0000011', '10', '000', '011', '011', '011', '000', '010', '010', '010']
000
Monday
000
010
['010', '0', '0001', '0100110', '00', '001', '001', '010', '1', '010', '010', '000', '0']
001
Tuesday
010
010
['101', '0', '0001', '0110110', '01', '011', '100', '100', '0', '011', '001', '101', '0']
011
Thursday
011
001
['111', '0', '0001', '0000010', '00', '010', '101', '111', '1', '011', '010', '000', '0']
010
Wednesday
011
010
['011', '1', '0001', '1000100', '01', '000', '100', '100', '100', '010', '100', '100', '100']
000
Monda

In [ ]:
import pandas as pd

# Define the days and time slots
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
time_slots = [
    '8:00 - 9:20',
    '9:35 - 10:55',
    '11:10 -12:30',
    '12:45 - 2:05',
    '2:20 - 3:40',
    '3:55 - 5:15'
]

# Define the rooms
rooms = ['C-401', 'C-402', 'C-403', 'C-404', 'C-405', 'C-406', 'Rawal-1', 'Rawal-2']

# Your provided schedule
schedule = [
    ['COAL lab', 'Lab', 'Section A', 92, 'Ms. Bushra', 'Friday', '9:35 - 10:55', 'C-403', '120', 'Friday', '11:10 -12:30', 'C-405', '120'],
    ['PF', 'Theory', 'Section A', 21, 'Ms. Bushra', 'Thursday', '9:35 - 10:55', 'C-402', '120', 'Tuesday', '9:35 - 10:55', 'C-403', '120'],
    # Add more entries as needed
]

# Create empty tables for each day
tables = {}
for day in days:
    tables[day] = pd.DataFrame(index=rooms, columns=time_slots)

# Fill in the tables with section and course information
for entry in decoded_chromosome:
    course_name = entry[0]
    section = entry[2]
    for i in range(5, len(entry), 4):  # Increment by 4 to skip the duration
        day = entry[i]
        start_time = entry[i+1]
        room = entry[i+2]
        tables[day].loc[room, start_time] = f'{course_name} ({section})'

# Display the tables as DataFrames
for day in days:
    print(f"Day: {day}")
    display(tables[day])
    print("\n\n")


Day: Monday


,8:00 - 9:20,9:35 - 10:55,11:10 -12:30,12:45 - 2:05,2:20 - 3:40,3:55 - 5:15
C-401,PF (Section N),PF (Section G),NaN,PF (Section G),NaN,NaN
C-402,NaN,NaN,NaN,NaN,AP (Section L),Discrete Structures (Section K)
C-403,NaN,Pakistan Studies (Section G),NaN,AP (Section G),NaN,NaN
C-404,PF (Section E),NaN,NaN,NaN,COAL (Section M),COAL lab (Section F)
C-405,AP (Section R),NaN,PF (Section M),Discrete Structures (Section G),PF lab (Section J),COAL lab (Section N)
C-406,NaN,PF lab (Section C),Pakistan Studies (Section R),AP (Section Z),NaN,NaN
Rawal-1,AP (Section L),NaN,NaN,NaN,NaN,NaN
Rawal-2,NaN,NaN,NaN,NaN,NaN,Discrete Structures (Section N)





Day: Tuesday


,8:00 - 9:20,9:35 - 10:55,11:10 -12:30,12:45 - 2:05,2:20 - 3:40,3:55 - 5:15
C-401,PF (Section J),NaN,NaN,NaN,NaN,NaN
C-402,Calculus (Section J),NaN,AP (Section H),NaN,Discrete Structures (Section Z),Pakistan Studies (Section K)
C-403,NaN,COAL lab (Section I),COAL lab (Section F),PF lab (Section C),NaN,NaN
C-404,NaN,NaN,NaN,NaN,NaN,NaN
C-405,PF lab (Section D),AP (Section H),PF (Section N),COAL (Section H),NaN,COAL lab (Section Z)
C-406,NaN,NaN,COAL (Section E),PF lab (Section M),NaN,NaN
Rawal-1,NaN,NaN,PF (Section H),NaN,COAL (Section M),Pakistan Studies (Section R)
Rawal-2,NaN,AP (Section E),Calculus (Section Z),NaN,NaN,NaN





Day: Wednesday


,8:00 - 9:20,9:35 - 10:55,11:10 -12:30,12:45 - 2:05,2:20 - 3:40,3:55 - 5:15
C-401,Pakistan Studies (Section H),COAL lab (Section L),Discrete Structures (Section B),NaN,NaN,COAL lab (Section C)
C-402,NaN,NaN,NaN,NaN,AP (Section L),COAL lab (Section L)
C-403,NaN,Pakistan Studies (Section F),Calculus (Section N),NaN,NaN,NaN
C-404,COAL (Section H),Calculus (Section J),NaN,PF lab (Section J),NaN,NaN
C-405,NaN,NaN,PF lab (Section L),NaN,NaN,NaN
C-406,COAL lab (Section L),NaN,NaN,Discrete Structures (Section Z),NaN,NaN
Rawal-1,NaN,NaN,NaN,NaN,NaN,NaN
Rawal-2,NaN,NaN,NaN,Discrete Structures (Section D),NaN,COAL (Section G)





Day: Thursday


,8:00 - 9:20,9:35 - 10:55,11:10 -12:30,12:45 - 2:05,2:20 - 3:40,3:55 - 5:15
C-401,PF lab (Section L),NaN,AP (Section B),Discrete Structures (Section N),Pakistan Studies (Section I),NaN
C-402,PF (Section E),Calculus (Section K),NaN,NaN,PF lab (Section R),COAL lab (Section M)
C-403,AP (Section R),Calculus (Section R),NaN,NaN,NaN,NaN
C-404,Discrete Structures (Section A),Pakistan Studies (Section N),NaN,NaN,NaN,Calculus (Section Z)
C-405,NaN,NaN,Pakistan Studies (Section N),NaN,NaN,COAL (Section D)
C-406,NaN,Calculus (Section F),NaN,COAL lab (Section Z),NaN,COAL lab (Section C)
Rawal-1,COAL lab (Section Z),NaN,NaN,Pakistan Studies (Section I),NaN,NaN
Rawal-2,NaN,NaN,NaN,NaN,Discrete Structures (Section G),COAL (Section E)





Day: Friday


,8:00 - 9:20,9:35 - 10:55,11:10 -12:30,12:45 - 2:05,2:20 - 3:40,3:55 - 5:15
C-401,Pakistan Studies (Section J),NaN,NaN,NaN,NaN,NaN
C-402,NaN,NaN,NaN,NaN,AP (Section Z),AP (Section I)
C-403,NaN,PF (Section K),NaN,NaN,NaN,NaN
C-404,NaN,NaN,NaN,NaN,NaN,NaN
C-405,NaN,NaN,PF lab (Section R),NaN,NaN,NaN
C-406,NaN,NaN,NaN,PF (Section M),NaN,NaN
Rawal-1,NaN,NaN,NaN,NaN,NaN,NaN
Rawal-2,NaN,NaN,NaN,NaN,NaN,NaN
